In [ ]:
using PerlaTonettiWaugh, Plots, BenchmarkTools

In [ ]:
z_min = 0.0 
z_max = 5.0
M = 1000
z_grid = range(z_min, stop = z_max, length = M) # Since we only care about the grid. 

# Time
tstops_min_Δ_val = 1e-3 # minimum distance between tstops to be used for DE solvers.
# Define common objects. 
d_0 = 5
d_T = 2.3701
params = (ρ = 0.02, σ = 4.2508, N = 10, θ = 5.1269, γ = 1.00, κ = 0.013, ζ = 1, η = 0, Theta = 1, χ = 1/(2.1868), υ = 0.0593, μ = 0, δ = 0.053, d = 2.3701) # Baselines per Jesse. 
stationary_sol = stationary_numerical(params, z_grid) # solution at t = T
println("g = $(stationary_sol.g), z_hat = $(stationary_sol.z_hat), Ω = $(stationary_sol.Ω)")

In [ ]:
T = 5.0
settings = (z = z_grid, tstops = 0:T, Δ_E = 1e-4)
Ω(t) = stationary_sol.Ω
stationary_sol.Ω
Δ_E = 1e-4
E(t) = (log(Ω(t+Δ_E)) - log(Ω(t-Δ_E)))/(2*Δ_E) + params.δ

Set ups and test f! first to see if residuals are reasonable at `T`

In [ ]:
# solve for stationary solution at t = 0
params_0 = merge(params, (d = d_0,)) # parameters to be used at t = 0
params_T = merge(params, (d = d_T,)) # parameters to be used at t = T

stationary_sol_0 = stationary_numerical(params_0, z_grid) # solution at t = 0
Ω_0 = stationary_sol_0.Ω

# get settings
settings = (z = z_grid, tstops = 0:1e-3:T, Δ_E = 1e-04)

In [ ]:
# #Testing the f!
du = zeros(M)
u = [stationary_sol.v_tilde; stationary_sol.g; stationary_sol.z_hat]
resid = similar(u)
du = zeros(M+2)
t = T
f! = solve_dynamics(params_T, stationary_sol, settings, T, Ω; stopwithf! = true)
@btime f!($resid,$du,$u,[ ],$t)

In [ ]:
resid

Now let's solve the problem and draw the plots!

In [ ]:
# solve the dynamics!
solved_tuple = solve_dynamics(params_T, stationary_sol, settings, T, Ω)
solved = solved_tuple.results
v_t0 = solved_tuple.sol.u[1][1:M]
v0 = solved[:v_0]

# save v0 and v_hat_t0
v_hat_t0 = map(z -> exp((params.σ-1)*z), z_grid) .* v_t0

In [ ]:
plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)

In [ ]:
plot(solved.t, solved.g, label = "g", lw = 3)

In [ ]:
plot(solved.t, solved.z_hat, label = "z_hat", lw = 3)

In [ ]:
plot(solved.t, solved.S, label = "S", lw = 3)

In [ ]:
plot(solved.t, solved.L_tilde, label = "L_tilde", lw = 3)

In [ ]:
plot(solved.t, solved.z_bar, label = "z_bar", lw = 3)

In [ ]:
plot(solved.t, solved.π_min, label = "pi_min", lw = 3)

In [ ]:
plot(solved.t, solved.λ_ii, label = "lambda_ii", lw = 3)

In [ ]:
plot(solved.t, solved.c, label = "c", lw = 3)

In [ ]:
plot(solved.t, solved.E, label = "E", lw = 3)

In [ ]:
plot(z_grid, v_hat_t0, label = "v_hat at t = 0", lw = 3)

In [ ]:
plot(solved.t, solved.v_0, label = "v0", lw = 3)